# load_saved_model
----

Written in the Python 3.7.9 Environment with the following package versions

    * joblib 1.0.1
    * numpy 1.19.5
    * pandas 1.3.1
    * scikit-learn 0.24.2
    * tensorflow 2.5.0

By Nicole Lund 

This Jupyter Notebook loads the best fit SVC model for Exoplanet classification from Kepler Exoplanet study data.

Column descriptions can be found at https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html 

**Source Data**

The source data used was provided by University of Arizona's Data Analytics homework assignment. Their data was derived from https://www.kaggle.com/nasa/kepler-exoplanet-search-results?select=cumulative.csv

The full data set was released by NASA at
https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=koi

In [1]:
# Import Dependencies

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Data manipulation
import numpy as np
import pandas as pd
from statistics import mean
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

# Parameter Selection
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Model Development
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Model Metrics
from sklearn.metrics import classification_report

# Save/load files
from tensorflow.keras.models import load_model
import joblib

# # Ignore deprecation warnings
# import warnings
# warnings.simplefilter('ignore', FutureWarning)

In [2]:
# Set the seed value for the notebook, so the results are reproducible
from numpy.random import seed
seed(1)

# Read the CSV and Perform Basic Data Cleaning

In [3]:
# Import data
df = pd.read_csv("b_source_data/exoplanet_data.csv")
# print(df.info())

# Drop columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop rows containing null values
df = df.dropna()

# Display data info
print(df.info())
print(df.head())
print(df.koi_disposition.unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

In [4]:
# Rename "FALSE POSITIVE" disposition values
df.koi_disposition = df.koi_disposition.str.replace(' ','_')
print(df.koi_disposition.unique())

['CONFIRMED' 'FALSE_POSITIVE' 'CANDIDATE']


# Select features

In [5]:
# Split dataframe into X and y

# Select features to analyze in X
select_option = 1

if select_option == 1:
    # Option 1: Choose all features
    X = df.drop("koi_disposition", axis=1)

elif select_option == 2:
    # Option 2: Choose all features that are not associated with error measurements
    X = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration','koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]

elif select_option == 3:
    # Option 3: Choose features from Decision Tree and Random Forest assessment.
    tree_features = ['koi_fpflag_nt', 'koi_fpflag_co', 'koi_fpflag_ss', 'koi_model_snr']
    forest_features = ['koi_fpflag_co', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr', 'koi_prad']
    X = df[set(tree_features + forest_features)]

# Define y
y = df["koi_disposition"]

print(X.shape, y.shape)

(6991, 40) (6991,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
# Split X and y into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [7]:
# Display training data
X_test.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
4982,1,0,0,0,88.523517,2.337000e-03,-2.337000e-03,189.275600,0.023200,-0.023200,...,-75,4.782,0.060,-0.070,0.462,0.050,-0.070,295.57629,48.897861,15.841
4866,0,0,0,0,102.493119,1.322000e-03,-1.322000e-03,166.009400,0.010500,-0.010500,...,-140,4.583,0.024,-0.117,0.791,0.132,-0.055,297.66437,46.944820,15.142
2934,0,0,1,0,1.181165,3.890000e-06,-3.890000e-06,132.425220,0.002760,-0.002760,...,-154,4.582,0.048,-0.078,0.747,0.098,-0.066,291.34879,51.256119,15.971
5007,0,1,0,0,8.412009,2.360000e-07,-2.360000e-07,139.648520,0.000024,-0.000024,...,-206,4.225,0.185,-0.185,1.314,0.373,-0.305,291.84970,37.738621,13.551
3869,0,1,1,1,1.195515,5.730000e-07,-5.730000e-07,132.303851,0.000410,-0.000410,...,-182,4.271,0.220,-0.180,1.155,0.331,-0.271,297.54337,41.947979,13.998


# Pre-processing

In [8]:
# Scale the data with MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# One-Hot-Encode the y data

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
print('Unique KOI Disposition Values')
print(y.unique())
print('-----------')
print('Sample KOI Disposition Values and Encoding')
print(y_test[:5])
print(y_test_categorical[:5])

Unique KOI Disposition Values
['CONFIRMED' 'FALSE_POSITIVE' 'CANDIDATE']
-----------
Sample KOI Disposition Values and Encoding
4982    FALSE_POSITIVE
4866         CANDIDATE
2934    FALSE_POSITIVE
5007    FALSE_POSITIVE
3869    FALSE_POSITIVE
Name: koi_disposition, dtype: object
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


# Load the saved model and grid



In [11]:
model = joblib.load('bestFit_SVC_model.sav')
grid = joblib.load('bestFit_SVC_grid.sav')

# Print scoring results of the trained model

In [12]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.880032699775189
Testing Data Score: 0.8746425166825548


In [13]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.8753276892451083


In [14]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [15]:
print(classification_report(y_test, predictions,
                            target_names=y.unique()))

                precision    recall  f1-score   support

     CONFIRMED       0.81      0.65      0.72       504
FALSE_POSITIVE       0.74      0.85      0.79       574
     CANDIDATE       0.98      1.00      0.99      1020

      accuracy                           0.87      2098
     macro avg       0.85      0.83      0.84      2098
  weighted avg       0.88      0.87      0.87      2098

